In [37]:
%pip install xgboost

import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from scipy import stats
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
!pip install statsmodels
import statsmodels.api as sm  
from statsmodels.stats.outliers_influence import variance_inflation_factor 






Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 4.0 MB/s eta 0:00:00a 0:00:01


In [27]:
heart_df = pd.read_csv('/Users/oniayotunde/ML/Machine-learning-Projects/mini_projects/heart_attack_youth_vs_adult_nigeria.csv')
heart_df.head(10)

,State,Age_Group,Gender,BMI,Smoking_Status,Alcohol_Consumption,Exercise_Frequency,Hypertension,Diabetes,Cholesterol_Level,Family_History,Stress_Level,Diet_Type,Heart_Attack_Severity,Hospitalized,Survived,Income_Level,Urban_Rural,Employment_Status
0,Ondo,Youth,Female,34.5,Non-Smoker,High,Occasionally,No,Yes,Borderline,Yes,Low,Unhealthy,Mild,No,Yes,Medium,Urban,Employed
1,FCT,Youth,Male,15.2,Non-Smoker,High,Occasionally,No,Yes,High,No,Moderate,Unhealthy,Mild,Yes,No,High,Rural,Employed
2,Borno,Youth,Female,25.0,Non-Smoker,High,Weekly,No,Yes,High,No,Moderate,Mixed,Moderate,Yes,No,Medium,Rural,Student
3,Katsina,Youth,Male,19.7,Non-Smoker,High,Occasionally,No,No,High,Yes,High,Healthy,Mild,Yes,No,Low,Rural,Unemployed
4,Kaduna,Adult,Female,35.6,Non-Smoker,Low,Rarely,No,Yes,High,Yes,High,Healthy,Moderate,No,Yes,Low,Rural,Student
5,Kogi,Youth,Female,38.8,Non-Smoker,NaN,Weekly,Yes,Yes,High,No,Low,Mixed,Moderate,Yes,Yes,Low,Rural,Employed
6,Ebonyi,Youth,Female,23.2,Smoker,Moderate,Daily,Yes,No,Normal,Yes,Low,Healthy,Severe,Yes,No,High,Rural,Unemployed
7,Ebonyi,Youth,Female,20.3,Smoker,NaN,Daily,Yes,No,Normal,No,Moderate,Mixed,Mild,Yes,Yes,Low,Urban,Employed
8,Kwara,Youth,Male,20.4,Smoker,Moderate,Weekly,No,No,Normal,Yes,Low,Healthy,Moderate,Yes,Yes,Low,Urban,Unemployed
9,Yobe,Adult,Male,30.2,Smoker,Moderate,Weekly,No,No,Normal,No,Low,Unhealthy,Severe,No,Yes,Low,Rural,Unemployed


In [28]:
heart_df['Alcohol_Consumption'].unique() #nan here means that the person does not drink alcohol, so we will replace it with No
heart_df['Alcohol_Consumption'] = heart_df['Alcohol_Consumption'].fillna('None')
heart_df['Alcohol_Consumption'].unique()

#the state columns doesnt seem to be useful for our analysis, so we will drop it
heart_df.drop('State', axis=1, inplace=True)
heart_df.head(10)

#model assumptions?? for the categorical variables, check for multicollinearity 
#and for the numerical variables, check for normality and homoscedasticity



,Age_Group,Gender,BMI,Smoking_Status,Alcohol_Consumption,Exercise_Frequency,Hypertension,Diabetes,Cholesterol_Level,Family_History,Stress_Level,Diet_Type,Heart_Attack_Severity,Hospitalized,Survived,Income_Level,Urban_Rural,Employment_Status
0,Youth,Female,34.5,Non-Smoker,High,Occasionally,No,Yes,Borderline,Yes,Low,Unhealthy,Mild,No,Yes,Medium,Urban,Employed
1,Youth,Male,15.2,Non-Smoker,High,Occasionally,No,Yes,High,No,Moderate,Unhealthy,Mild,Yes,No,High,Rural,Employed
2,Youth,Female,25.0,Non-Smoker,High,Weekly,No,Yes,High,No,Moderate,Mixed,Moderate,Yes,No,Medium,Rural,Student
3,Youth,Male,19.7,Non-Smoker,High,Occasionally,No,No,High,Yes,High,Healthy,Mild,Yes,No,Low,Rural,Unemployed
4,Adult,Female,35.6,Non-Smoker,Low,Rarely,No,Yes,High,Yes,High,Healthy,Moderate,No,Yes,Low,Rural,Student
5,Youth,Female,38.8,Non-Smoker,None,Weekly,Yes,Yes,High,No,Low,Mixed,Moderate,Yes,Yes,Low,Rural,Employed
6,Youth,Female,23.2,Smoker,Moderate,Daily,Yes,No,Normal,Yes,Low,Healthy,Severe,Yes,No,High,Rural,Unemployed
7,Youth,Female,20.3,Smoker,None,Daily,Yes,No,Normal,No,Moderate,Mixed,Mild,Yes,Yes,Low,Urban,Employed
8,Youth,Male,20.4,Smoker,Moderate,Weekly,No,No,Normal,Yes,Low,Healthy,Moderate,Yes,Yes,Low,Urban,Unemployed
9,Adult,Male,30.2,Smoker,Moderate,Weekly,No,No,Normal,No,Low,Unhealthy,Severe,No,Yes,Low,Rural,Unemployed


In [46]:
#I want to run this model on logistice regression and the column heart attack risk will be our target var 
# I want to know what I can assume for me to use this model 
# given that the model was collected across diffrent states and people, we would say that 
# it satisfies the assumption of independence
# linearity of logit will be checked after the model has been run


# multicollinearity

#first make a copy of df so that we can encode the categorical variables

heart_df_copy = heart_df.copy()
heart_df_copy.columns


#we will encode the categorical)(ordinal variables) variables
ordinal_cat = ['Alcohol_Consumption',
'Exercise_Frequency',
'Cholesterol_Level',
'Stress_Level',
'Income_Level',]

for column in heart_df[ordinal_cat].columns:
    column_values = heart_df[column].unique()
    print(f'{column}: {column_values}')


categories = [['None','Low','Moderate','High' ],
['Rarely', 'Daily','Weekly','Occasionally'],
['Normal','Borderline','High' ],
['Low','Moderate','High'],
['Low','Medium','High']]





nominal_cat = heart_df_copy.columns.difference(ordinal_cat)
nominal_cat

#remove BMI from nominal_cat
nominal_cat = nominal_cat.drop('BMI')
nominal_cat

#we will encode the ordinal variables using the ordinal encoder
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder(categories=categories)
heart_df_copy[ordinal_cat] = encoder.fit_transform(heart_df_copy[ordinal_cat])
heart_df_copy.head(10)

#we will encode the nominal variables using the one hot encoder 
heart_df_copy = pd.get_dummies(heart_df_copy, columns=nominal_cat, drop_first=True)
heart_df_copy.head(10)


scaler = StandardScaler()
X_scaled = scaler.fit_transform(heart_df_copy)
X_scaled = pd.DataFrame(X_scaled, columns=heart_df_copy.columns)


#add constant to the scaled data
X_scaled = sm.add_constant(X_scaled)

#Calculate VIF
vif_data = pd.DataFrame()
vif_data['Feature'] = X_scaled.columns
vif_data['VIF'] = [variance_inflation_factor(X_scaled.values, i) 
                   for i in range(X_scaled.shape[1])]

#Display results
print("VIF Results:")
print(vif_data) #we can see that all the VIF values are less than 5, so we can say that there is no multicollinearity




Alcohol_Consumption: ['High' 'Low' 'None' 'Moderate']
Exercise_Frequency: ['Occasionally' 'Weekly' 'Rarely' 'Daily']
Cholesterol_Level: ['Borderline' 'High' 'Normal']
Stress_Level: ['Low' 'Moderate' 'High']
Income_Level: ['Medium' 'High' 'Low']


/Users/oniayotunde/miniforge3/envs/ml_env/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


VIF Results:
                           Feature       VIF
0                            const  1.000000
1                              BMI  1.000038
2              Alcohol_Consumption  1.000061
3               Exercise_Frequency  1.000067
4                Cholesterol_Level  1.000060
5                     Stress_Level  1.000070
6                     Income_Level  1.000074
7                  Age_Group_Youth  1.000053
8                     Diabetes_Yes  1.000088
9                  Diet_Type_Mixed  1.331769
10             Diet_Type_Unhealthy  1.331754
11       Employment_Status_Student  1.331859
12    Employment_Status_Unemployed  1.331920
13              Family_History_Yes  1.000040
14                     Gender_Male  1.000053
15  Heart_Attack_Severity_Moderate  1.334546
16    Heart_Attack_Severity_Severe       inf
17  Heart_Attack_Severity_Binary_1       inf
18                Hospitalized_Yes  1.000057
19                Hypertension_Yes  1.000074
20           Smoking_Status_Smoker  1.0000

In [ ]:
heart_df_copy.head(10)
heart_df_copy.drop('Heart_Attack_Severity_Binary_1', axis=1, inplace=True)



KeyError: "['Heart_Attack_Severity_Binary_1'] not found in axis"

In [65]:
heart_df_copy.head(10)

,BMI,Alcohol_Consumption,Exercise_Frequency,Cholesterol_Level,Stress_Level,Income_Level,Age_Group_Youth,Diabetes_Yes,Diet_Type_Mixed,Diet_Type_Unhealthy,...,Employment_Status_Unemployed,Family_History_Yes,Gender_Male,Heart_Attack_Severity_Moderate,Heart_Attack_Severity_Severe,Hospitalized_Yes,Hypertension_Yes,Smoking_Status_Smoker,Survived_Yes,Urban_Rural_Urban
0,34.5,3.0,3.0,1.0,0.0,1.0,True,True,False,True,...,False,True,False,False,False,False,False,False,True,True
1,15.2,3.0,3.0,2.0,1.0,2.0,True,True,False,True,...,False,False,True,False,False,True,False,False,False,False
2,25.0,3.0,2.0,2.0,1.0,1.0,True,True,True,False,...,False,False,False,True,False,True,False,False,False,False
3,19.7,3.0,3.0,2.0,2.0,0.0,True,False,False,False,...,True,True,True,False,False,True,False,False,False,False
4,35.6,1.0,0.0,2.0,2.0,0.0,False,True,False,False,...,False,True,False,True,False,False,False,False,True,False
5,38.8,0.0,2.0,2.0,0.0,0.0,True,True,True,False,...,False,False,False,True,False,True,True,False,True,False
6,23.2,2.0,1.0,0.0,0.0,2.0,True,False,False,False,...,True,True,False,False,True,True,True,True,False,False
7,20.3,0.0,1.0,0.0,1.0,0.0,True,False,True,False,...,False,False,False,False,False,True,True,True,True,True
8,20.4,2.0,2.0,0.0,0.0,0.0,True,False,False,False,...,True,True,True,True,False,True,False,True,True,True
9,30.2,2.0,2.0,0.0,0.0,0.0,False,False,False,True,...,True,False,True,False,True,False,False,True,True,False
